In [1]:
# clone the git repo
!git clone https://github.com/Nikhil29/Hackathon.git

Cloning into 'Hackathon'...
error: failed to read delta-pack base object a160d0387a126e3d527b3e9530a0154cb52e47f2
fatal: unpack-objects failed


In [ ]:
# set the folder to use the ython files etc.
import sys
sys.path.append('/content/Hackathon')

In [ ]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for displaying DataFrames

import matplotlib.pyplot as plt
import seaborn as sns

# Import supplementary visualization code visuals.py from project root folder
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

In [ ]:
# Load the Red Wines dataset
path = '/content/Hackathon/data/01.xls';
data = pd.read_excel(path)

# Display the first five records
display(data.head(n=5))